In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import torch.optim as optim

In [2]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header= None, engine = 'python', encoding = 'latin-1')

In [3]:
users = pd.read_csv('ml-1m/users.dat', sep = '::', header= None, engine = 'python', encoding = 'latin-1')

In [4]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header= None, engine = 'python', encoding = 'latin-1')

In [5]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')

In [6]:
training_set = np.array(training_set, dtype = 'int')

In [7]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')

In [8]:
test_set = np.array(test_set, dtype = 'int')

In [9]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [10]:
def convert(data):
    new_data = []
    for id_users in range(1,nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [11]:
training_set = convert(training_set)
test_set = convert(test_set)

In [12]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [13]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
    
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [14]:
nb_epoch = 200
for epoch in range(1, nb_epoch+1):
    train_loss = 0.
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+'  Loss: '+str(train_loss/s))

epoch: 1  Loss: tensor(1.7715)
epoch: 2  Loss: tensor(1.0968)
epoch: 3  Loss: tensor(1.0533)
epoch: 4  Loss: tensor(1.0383)
epoch: 5  Loss: tensor(1.0308)
epoch: 6  Loss: tensor(1.0266)
epoch: 7  Loss: tensor(1.0239)
epoch: 8  Loss: tensor(1.0218)
epoch: 9  Loss: tensor(1.0206)
epoch: 10  Loss: tensor(1.0194)
epoch: 11  Loss: tensor(1.0187)
epoch: 12  Loss: tensor(1.0186)
epoch: 13  Loss: tensor(1.0178)
epoch: 14  Loss: tensor(1.0175)
epoch: 15  Loss: tensor(1.0173)
epoch: 16  Loss: tensor(1.0169)
epoch: 17  Loss: tensor(1.0165)
epoch: 18  Loss: tensor(1.0165)
epoch: 19  Loss: tensor(1.0161)
epoch: 20  Loss: tensor(1.0162)
epoch: 21  Loss: tensor(1.0158)
epoch: 22  Loss: tensor(1.0158)
epoch: 23  Loss: tensor(1.0157)
epoch: 24  Loss: tensor(1.0158)
epoch: 25  Loss: tensor(1.0156)
epoch: 26  Loss: tensor(1.0154)
epoch: 27  Loss: tensor(1.0156)
epoch: 28  Loss: tensor(1.0149)
epoch: 29  Loss: tensor(1.0133)
epoch: 30  Loss: tensor(1.0118)
epoch: 31  Loss: tensor(1.0096)
epoch: 32  Loss: 

In [15]:
test_loss = 0.
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[(target == 0).unsqueeze(0)] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('Loss: '+str(test_loss/s))

Loss: tensor(0.9597)


/Users/vishaljha/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
